In [1]:
!pip install durable_rules

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 820.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for durable_rules: filename=durable_rules-2.0.28-cp310-cp310-linux_x86_64.whl size=199324 sha256=3dd234e0cf10542f94d213749c3473e7a449d1640c8d478bb0104074bcb13c1c
  Stored in directory: /root/.cache/pip/wheels/f6/b4/6c/ee50ba2f212686ca9c6fdae66f5030f2f37cc9178a38503f69
Successfully built durable_rules


In [2]:
from durable.lang import *

with ruleset('Device_status'):
    @when_all((m.predicate == 'status') & (m.object == '0'))
    def normal(c):
        c.assert_fact({'subject': c.m.subject, 'predicate': 'finalStatus', 'object': 'normal'})

    @when_all((m.predicate == 'status') & (m.object == '1'))
    def abnormal(c):
        c.assert_fact({'subject': c.m.subject, 'predicate': 'finalStatus', 'object': 'abnormal'})

    @when_all((m.predicate == 'status') & (m.object == '2'))
    def communication_failure(c):
        c.assert_fact({'subject': c.m.subject, 'predicate': 'finalStatus', 'object': 'communication_failure'})

    @when_all((m.predicate == 'status') & (m.object == '4'))
    def power_disconnection(c):
        c.assert_fact({'subject': c.m.subject, 'predicate': 'finalStatus', 'object': 'power_disconnection'})

    @when_all((m.predicate == 'status') & (m.object == '8'))
    def under_maintenance(c):
        c.assert_fact({'subject': c.m.subject, 'predicate': 'finalStatus', 'object': 'under_maintenance'})

    @when_all((m.predicate == 'fa_value') & (m.object == '3.9'))
    def abnormal_value(c):
        c.assert_fact({'subject': c.m.subject, 'predicate': 'finalStatus', 'object': 'abnormal(value)'})

    @when_all((m.predicate == 'fa_value') & (m.object == '4.0'))
    def normal_value(c):
        c.assert_fact({'subject': c.m.subject, 'predicate': 'finalStatus', 'object': 'normal(value)'})

    @when_all(c.first << (m.predicate == 'status') & (m.object == '0'),
            (m.predicate == 'runStatus') & (m.object == '0') & (m.subject == c.first.subject))
    def nonOperational(c):
      c.assert_fact({'subject': c.m.subject, 'predicate': 'Operational status', 'object': 'non-operational'})

    @when_all(c.first << (m.predicate == 'status') & (m.object == '0'),
              (m.predicate == 'runStatus') & (m.object == '1') & (m.subject == c.first.subject))
    def operational(c):
      c.assert_fact({'subject': c.m.subject, 'predicate': 'Operational status', 'object': 'operational'})

    @when_all(+m.subject)
    def output(c):
        print('Fact: {0} {1} {2}'.format(c.m.subject, c.m.predicate, c.m.object), '\n')

In [3]:
# 장치가 정상 상태이지만 운영되지 않음
assert_fact('Device_status', {'subject': 'device1', 'predicate': 'status', 'object': '0'})
assert_fact('Device_status', {'subject': 'device1', 'predicate': 'runStatus', 'object': '0'})

# 장치가 정상 상태이며 운영 중
assert_fact('Device_status', {'subject': 'device2', 'predicate': 'status', 'object': '0'})
assert_fact('Device_status', {'subject': 'device2', 'predicate': 'runStatus', 'object': '1'})


Fact: device1 finalStatus normal 

Fact: device1 status 0 

Fact: device1 runStatus 0 

Fact: device2 finalStatus normal 

Fact: device2 status 0 

Fact: device2 runStatus 1 



{'sid': '0',
 'id': 'sid-0',
 '$s': 1,
 'exception': 'exception caught \'NoneType\' object has no attribute \'subject\', traceback [\'  File "/usr/local/lib/python3.10/dist-packages/durable/engine.py", line 241, in run\\n    self._func(c)\\n\', \'  File "<ipython-input-2-a71f4f47d99f>", line 40, in operational\\n    c.assert_fact({\\\'subject\\\': c.m.subject, \\\'predicate\\\': \\\'Operational status\\\', \\\'object\\\': \\\'operational\\\'})\\n\']'}

In [4]:
assert_fact('Device_status', {'subject': 'device3', 'predicate': 'status', 'object': '1'})
assert_fact('Device_status', {'subject': 'device4', 'predicate': 'status', 'object': '0'})

Fact: device3 finalStatus abnormal 

Fact: device3 status 1 

Fact: device4 finalStatus normal 

Fact: device4 status 0 



{'sid': '0',
 'id': 'sid-0',
 '$s': 1,
 'exception': 'exception caught \'NoneType\' object has no attribute \'subject\', traceback [\'  File "/usr/local/lib/python3.10/dist-packages/durable/engine.py", line 241, in run\\n    self._func(c)\\n\', \'  File "<ipython-input-2-a71f4f47d99f>", line 40, in operational\\n    c.assert_fact({\\\'subject\\\': c.m.subject, \\\'predicate\\\': \\\'Operational status\\\', \\\'object\\\': \\\'operational\\\'})\\n\']'}